## LightGBM with Simple Feature Engineering
To Do:
- [x] Add 4 weeks of data
- [x] Add 8 weeks of data
- [x] Add feature importance\selection 4 weeks of data
- [x] Add feature importance\selection 8 weeks of data
- [x] Add 4 monhts of data to perfrom 70/30 split (best results so far)
- [] Add hyperparameter tuning (next nb)

### 4 weeks of data

In [1]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89300,3.0,0.0,2.0,3.0,2.0,3.0,13.0,8.0,9.0,9.0,...,6.0,5.0,3.0,1.0,6.0,1.0,3.0,2022-12-27,265,3.0
89301,6.0,4.0,0.0,0.0,2.0,0.0,14.0,7.0,8.0,4.0,...,4.0,2.0,1.0,2.0,2.0,2.0,8.0,2022-12-28,265,1.0
89302,7.0,2.0,3.0,4.0,7.0,4.0,10.0,9.0,7.0,11.0,...,2.0,3.0,5.0,1.0,1.0,0.0,8.0,2022-12-29,265,3.0
89303,6.0,5.0,4.0,3.0,0.0,3.0,11.0,12.0,9.0,10.0,...,3.0,3.0,1.0,2.0,0.0,1.0,2.0,2022-12-30,265,7.0


In [2]:
from datetime import datetime
from src.data_split import train_test_split

# Split data into train and test sets
X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32595, 674)
y_train.shape=(32595,)
X_test.shape=(56710, 674)
y_test.shape=(56710,)


In [3]:
def average_rides_last_4_weeks(X: pd.DataFrame) -> pd.DataFrame:
    """
    Adds one column with the average rides from
    - 7 days ago
    - 14 days ago
    - 21 days ago
    - 28 days ago

    Parameters
    ----------
    X : pd.DataFrame
        Input data

    Returns
    -------
    pd.DataFrame
        Input data with one additional column      
    """
    X['average_rides_last_4_weeks'] = 0.25*(
        X[f'rides_previous_{7*24}_hour'] + \
        X[f'rides_previous_{2*7*24}_hour'] + \
        X[f'rides_previous_{3*7*24}_hour'] + \
        X[f'rides_previous_{4*7*24}_hour']
    )
    return X

In [10]:
# transform the data
from sklearn.preprocessing import FunctionTransformer

add_feature_average_rides_last_4_weeks = FunctionTransformer(
    average_rides_last_4_weeks,validate=False)

In [11]:
add_feature_average_rides_last_4_weeks.fit_transform(X_train)

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,average_rides_last_4_weeks
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.00
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.00
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.00
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.00
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32590,10.0,7.0,4.0,5.0,10.0,4.0,9.0,15.0,15.0,13.0,...,23.0,12.0,8.0,8.0,4.0,5.0,7.0,2022-05-27,265,7.00
32591,6.0,5.0,5.0,0.0,6.0,4.0,1.0,3.0,3.0,10.0,...,19.0,20.0,9.0,5.0,6.0,7.0,2.0,2022-05-28,265,7.25
32592,8.0,6.0,2.0,5.0,4.0,3.0,2.0,3.0,15.0,14.0,...,9.0,6.0,5.0,3.0,3.0,11.0,5.0,2022-05-29,265,8.75
32593,4.0,2.0,5.0,3.0,1.0,7.0,15.0,18.0,17.0,14.0,...,7.0,8.0,13.0,11.0,5.0,4.0,4.0,2022-05-30,265,6.00


In [12]:
from sklearn.base import BaseEstimator, TransformerMixin

class TemporalFeaturesEngineer(BaseEstimator, TransformerMixin):
    """
    Adds the following columns:
    - hour
    - day_of_week

    Parameters
    ----------
    BaseEstimator : [type]
        [description]
    TransformerMixin : [type]
        [description]

    Returns
    -------
    DataFrame with additional column
        Numeric column from datetime (day_of_week)
        Numeric column from datetime (hour) is DROPPED
    """

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_ = X.copy()

        # generate numeric columns from datetime
        X_['hour'] = X_['pickup_hour'].dt.hour
        X_['day_of_week'] = X_['pickup_hour'].dt.dayofweek

        return X_.drop(columns=['pickup_hour'])

In [13]:
add_temporal_features = TemporalFeaturesEngineer()
add_temporal_features.fit_transform(X_train)

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_location_id,average_rides_last_4_weeks,hour,day_of_week
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1,0.00,0,5
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1,0.00,0,6
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,1.0,2.0,0.0,0.0,0.0,0.0,1,0.00,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,1.0,0.0,0.0,1,0.00,0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0.00,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32590,10.0,7.0,4.0,5.0,10.0,4.0,9.0,15.0,15.0,13.0,...,12.0,8.0,8.0,4.0,5.0,7.0,265,7.00,0,4
32591,6.0,5.0,5.0,0.0,6.0,4.0,1.0,3.0,3.0,10.0,...,20.0,9.0,5.0,6.0,7.0,2.0,265,7.25,0,5
32592,8.0,6.0,2.0,5.0,4.0,3.0,2.0,3.0,15.0,14.0,...,6.0,5.0,3.0,3.0,11.0,5.0,265,8.75,0,6
32593,4.0,2.0,5.0,3.0,1.0,7.0,15.0,18.0,17.0,14.0,...,8.0,13.0,11.0,5.0,4.0,4.0,265,6.00,0,0


In [9]:
import lightgbm as lgb

In [14]:
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    add_feature_average_rides_last_4_weeks,
    add_temporal_features,
    lgb.LGBMRegressor()
)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x138288310>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor', LGBMRegressor())])

In [15]:
predictions = pipeline.predict(X_test)

from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=2.5947


### Feature Importance + Feature Engineering data - 4 weeks

In [33]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_regression

pipeline = make_pipeline(
    add_feature_average_rides_last_4_weeks,
    add_temporal_features,
    SelectKBest(f_regression, k=33), #f_regression is for regression, #k=33 is the num of features
    lgb.LGBMRegressor()
)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x138288310>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('selectkbest',
                 SelectKBest(k=33,
                             score_func=<function f_regression at 0x15f6739d0>)),
                ('lgbmregressor', LGBMRegressor())])

In [35]:
predictions = pipeline.predict(X_test)

from sklearn.metrics import mean_absolute_error
test_mae_importance = mean_absolute_error(y_test, predictions)
print(f'{test_mae_importance=:.4f}')

test_mae_importance=2.5695


### Summary 4 weeks

[w\o feature importance] 
- mae = 2.59
- feature_size = 676
- pipeline train time = 6.1 sec

[w feature importance] 
- test_mae_importance =2.5695 
- feature_size = 33
- pipeline train time = 2 sec

### 8 weeks of data

In [46]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data_2_months.parquet')
df

,rides_previous_1344_hour,rides_previous_1343_hour,rides_previous_1342_hour,rides_previous_1341_hour,rides_previous_1340_hour,rides_previous_1339_hour,rides_previous_1338_hour,rides_previous_1337_hour,rides_previous_1336_hour,rides_previous_1335_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2022-02-26,1,1.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,2022-02-27,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,3.0,2.0,0.0,2.0,0.0,0.0,0.0,2022-02-28,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-03-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,0.0,0.0,0.0,0.0,0.0,2022-03-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81880,4.0,6.0,0.0,1.0,1.0,7.0,22.0,18.0,25.0,17.0,...,6.0,5.0,3.0,1.0,6.0,1.0,3.0,2022-12-27,265,3.0
81881,2.0,6.0,4.0,2.0,4.0,10.0,16.0,17.0,20.0,12.0,...,4.0,2.0,1.0,2.0,2.0,2.0,8.0,2022-12-28,265,1.0
81882,5.0,1.0,3.0,1.0,2.0,8.0,18.0,9.0,18.0,19.0,...,2.0,3.0,5.0,1.0,1.0,0.0,8.0,2022-12-29,265,3.0
81883,8.0,6.0,3.0,3.0,2.0,8.0,15.0,16.0,8.0,8.0,...,3.0,3.0,1.0,2.0,0.0,1.0,2.0,2022-12-30,265,7.0


In [93]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(25175, 1346)
y_train.shape=(25175,)
X_test.shape=(56710, 1346)
y_test.shape=(56710,)


In [94]:
def average_rides_last_8_weeks(X: pd.DataFrame) -> pd.DataFrame:
    """
    Adds one column with the average rides from
    - 7 days ago
    - 14 days ago
    - 21 days ago
    - 28 days ago
    - 35 days ago
    - 42 days ago
    - 49 days ago
    - 56 days ago

    Parameters
    ----------
    X : pd.DataFrame
        Input data

    Returns
    -------
    pd.DataFrame
        Input data with one additional column      
    """
    X['average_rides_last_8_weeks'] = 0.125*(
        X[f'rides_previous_{7*24}_hour'] + \
        X[f'rides_previous_{2*7*24}_hour'] + \
        X[f'rides_previous_{3*7*24}_hour'] + \
        X[f'rides_previous_{4*7*24}_hour'] + \
        X[f'rides_previous_{5*7*24}_hour'] + \
        X[f'rides_previous_{6*7*24}_hour'] + \
        X[f'rides_previous_{7*7*24}_hour'] + \
        X[f'rides_previous_{8*7*24}_hour']
    )
    return X

In [95]:
# transform the data
from sklearn.preprocessing import FunctionTransformer

add_feature_average_rides_last_8_weeks = FunctionTransformer(
    average_rides_last_8_weeks,validate=False)

In [96]:
add_feature_average_rides_last_8_weeks.fit_transform(X_train)

,rides_previous_1344_hour,rides_previous_1343_hour,rides_previous_1342_hour,rides_previous_1341_hour,rides_previous_1340_hour,rides_previous_1339_hour,rides_previous_1338_hour,rides_previous_1337_hour,rides_previous_1336_hour,rides_previous_1335_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,average_rides_last_8_weeks
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2022-02-26,1,0.000
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,2022-02-27,1,0.000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,3.0,2.0,0.0,2.0,0.0,0.0,0.0,2022-02-28,1,0.000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-03-01,1,0.000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,0.0,0.0,0.0,0.0,0.0,2022-03-02,1,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25170,9.0,9.0,3.0,4.0,6.0,5.0,23.0,23.0,21.0,16.0,...,23.0,12.0,8.0,8.0,4.0,5.0,7.0,2022-05-27,265,7.000
25171,11.0,3.0,4.0,3.0,3.0,7.0,2.0,3.0,6.0,9.0,...,19.0,20.0,9.0,5.0,6.0,7.0,2.0,2022-05-28,265,8.250
25172,6.0,9.0,6.0,2.0,8.0,5.0,6.0,1.0,2.0,7.0,...,9.0,6.0,5.0,3.0,3.0,11.0,5.0,2022-05-29,265,8.375
25173,6.0,3.0,1.0,0.0,5.0,8.0,20.0,14.0,24.0,38.0,...,7.0,8.0,13.0,11.0,5.0,4.0,4.0,2022-05-30,265,5.500


In [97]:
from sklearn.base import BaseEstimator, TransformerMixin

class TemporalFeaturesEngineer(BaseEstimator, TransformerMixin):
    """
    Adds the following columns:
    - hour
    - day_of_week

    Parameters
    ----------
    BaseEstimator : [type]
        [description]
    TransformerMixin : [type]
        [description]

    Returns
    -------
    DataFrame with additional column
        Numeric column from datetime (day_of_week)
        Numeric column from datetime (hour) is DROPPED
    """

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_ = X.copy()

        # generate numeric columns from datetime
        X_['hour'] = X_['pickup_hour'].dt.hour
        X_['day_of_week'] = X_['pickup_hour'].dt.dayofweek
        
        return X_.drop(columns=['pickup_hour'])

In [98]:
add_temporal_features = TemporalFeaturesEngineer()
add_temporal_features.fit_transform(X_train)

,rides_previous_1344_hour,rides_previous_1343_hour,rides_previous_1342_hour,rides_previous_1341_hour,rides_previous_1340_hour,rides_previous_1339_hour,rides_previous_1338_hour,rides_previous_1337_hour,rides_previous_1336_hour,rides_previous_1335_hour,...,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_location_id,average_rides_last_8_weeks,hour,day_of_week
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,1,0.000,0,5
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1,0.000,0,6
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,2.0,0.0,2.0,0.0,0.0,0.0,1,0.000,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000,0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,1,0.000,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25170,9.0,9.0,3.0,4.0,6.0,5.0,23.0,23.0,21.0,16.0,...,12.0,8.0,8.0,4.0,5.0,7.0,265,7.000,0,4
25171,11.0,3.0,4.0,3.0,3.0,7.0,2.0,3.0,6.0,9.0,...,20.0,9.0,5.0,6.0,7.0,2.0,265,8.250,0,5
25172,6.0,9.0,6.0,2.0,8.0,5.0,6.0,1.0,2.0,7.0,...,6.0,5.0,3.0,3.0,11.0,5.0,265,8.375,0,6
25173,6.0,3.0,1.0,0.0,5.0,8.0,20.0,14.0,24.0,38.0,...,8.0,13.0,11.0,5.0,4.0,4.0,265,5.500,0,0


In [99]:
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    add_feature_average_rides_last_8_weeks,
    add_temporal_features,
    lgb.LGBMRegressor()
)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_8_weeks at 0x162067a60>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor', LGBMRegressor())])

In [100]:
predictions = pipeline.predict(X_test)

from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=2.7288


### Feature Importance + Feature Engineering data - 8 weeks

In [108]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_regression

pipeline = make_pipeline(
    add_feature_average_rides_last_8_weeks,
    add_temporal_features,
    SelectKBest(f_regression, k=40), #f_regression is for regression, #k=33 is the num of features
    lgb.LGBMRegressor()
)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_8_weeks at 0x162067a60>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('selectkbest',
                 SelectKBest(k=40,
                             score_func=<function f_regression at 0x15f6739d0>)),
                ('lgbmregressor', LGBMRegressor())])

In [109]:
predictions = pipeline.predict(X_test)

from sklearn.metrics import mean_absolute_error
test_mae_importance = mean_absolute_error(y_test, predictions)
print(f'{test_mae_importance=:.4f}')

test_mae_importance=2.7380


### Summary 8 weeks
[w\o feature importance] 
- mae = 2.72
- feature_size = 1348
- pipeline train time = 8.5 sec

[w feature importance] 
- test_mae_importance = 2.73
- feature_size = 40
- pipeline train time = 3 sec

### Experiments section
To Do:
- [x] Experiment with 16 weeks (4 months) of data - to get the 70\30 split of the train and test data

In [110]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data_2_months.parquet')
df

,rides_previous_1344_hour,rides_previous_1343_hour,rides_previous_1342_hour,rides_previous_1341_hour,rides_previous_1340_hour,rides_previous_1339_hour,rides_previous_1338_hour,rides_previous_1337_hour,rides_previous_1336_hour,rides_previous_1335_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2022-02-26,1,1.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,2022-02-27,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,3.0,2.0,0.0,2.0,0.0,0.0,0.0,2022-02-28,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-03-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,0.0,0.0,0.0,0.0,0.0,2022-03-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81880,4.0,6.0,0.0,1.0,1.0,7.0,22.0,18.0,25.0,17.0,...,6.0,5.0,3.0,1.0,6.0,1.0,3.0,2022-12-27,265,3.0
81881,2.0,6.0,4.0,2.0,4.0,10.0,16.0,17.0,20.0,12.0,...,4.0,2.0,1.0,2.0,2.0,2.0,8.0,2022-12-28,265,1.0
81882,5.0,1.0,3.0,1.0,2.0,8.0,18.0,9.0,18.0,19.0,...,2.0,3.0,5.0,1.0,1.0,0.0,8.0,2022-12-29,265,3.0
81883,8.0,6.0,3.0,3.0,2.0,8.0,15.0,16.0,8.0,8.0,...,3.0,3.0,1.0,2.0,0.0,1.0,2.0,2022-12-30,265,7.0


In [111]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 10, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(57505, 1346)
y_train.shape=(57505,)
X_test.shape=(24380, 1346)
y_test.shape=(24380,)


In [112]:
def average_rides_last_8_weeks(X: pd.DataFrame) -> pd.DataFrame:
    """
    Adds one column with the average rides from
    - 7 days ago
    - 14 days ago
    - 21 days ago
    - 28 days ago
    - 35 days ago
    - 42 days ago
    - 49 days ago
    - 56 days ago

    Parameters
    ----------
    X : pd.DataFrame
        Input data

    Returns
    -------
    pd.DataFrame
        Input data with one additional column      
    """
    X['average_rides_last_8_weeks'] = 0.125*(
        X[f'rides_previous_{7*24}_hour'] + \
        X[f'rides_previous_{2*7*24}_hour'] + \
        X[f'rides_previous_{3*7*24}_hour'] + \
        X[f'rides_previous_{4*7*24}_hour'] + \
        X[f'rides_previous_{5*7*24}_hour'] + \
        X[f'rides_previous_{6*7*24}_hour'] + \
        X[f'rides_previous_{7*7*24}_hour'] + \
        X[f'rides_previous_{8*7*24}_hour']
    )
    return X

In [113]:
# transform the data
from sklearn.preprocessing import FunctionTransformer

add_feature_average_rides_last_8_weeks = FunctionTransformer(
    average_rides_last_8_weeks,validate=False)

In [114]:
add_feature_average_rides_last_8_weeks.fit_transform(X_train)

,rides_previous_1344_hour,rides_previous_1343_hour,rides_previous_1342_hour,rides_previous_1341_hour,rides_previous_1340_hour,rides_previous_1339_hour,rides_previous_1338_hour,rides_previous_1337_hour,rides_previous_1336_hour,rides_previous_1335_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,average_rides_last_8_weeks
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2022-02-26,1,0.000
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,2022-02-27,1,0.000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,3.0,2.0,0.0,2.0,0.0,0.0,0.0,2022-02-28,1,0.000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-03-01,1,0.000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,0.0,0.0,0.0,0.0,0.0,2022-03-02,1,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57500,7.0,5.0,1.0,1.0,3.0,4.0,15.0,17.0,21.0,9.0,...,17.0,7.0,3.0,5.0,2.0,8.0,5.0,2022-09-26,265,4.000
57501,5.0,1.0,1.0,1.0,4.0,7.0,8.0,17.0,12.0,15.0,...,6.0,8.0,8.0,4.0,2.0,7.0,10.0,2022-09-27,265,3.875
57502,6.0,3.0,3.0,1.0,0.0,3.0,7.0,20.0,12.0,27.0,...,10.0,7.0,2.0,2.0,6.0,6.0,3.0,2022-09-28,265,4.125
57503,10.0,2.0,1.0,1.0,4.0,3.0,11.0,13.0,7.0,15.0,...,4.0,0.0,4.0,5.0,2.0,7.0,2.0,2022-09-29,265,7.375


In [115]:
from sklearn.base import BaseEstimator, TransformerMixin

class TemporalFeaturesEngineer(BaseEstimator, TransformerMixin):
    """
    Adds the following columns:
    - hour
    - day_of_week

    Parameters
    ----------
    BaseEstimator : [type]
        [description]
    TransformerMixin : [type]
        [description]

    Returns
    -------
    DataFrame with additional column
        Numeric column from datetime (day_of_week)
        Numeric column from datetime (hour) is DROPPED
    """

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_ = X.copy()

        # generate numeric columns from datetime
        X_['hour'] = X_['pickup_hour'].dt.hour
        X_['day_of_week'] = X_['pickup_hour'].dt.dayofweek
        
        return X_.drop(columns=['pickup_hour'])

In [116]:
add_temporal_features = TemporalFeaturesEngineer()
add_temporal_features.fit_transform(X_train)

,rides_previous_1344_hour,rides_previous_1343_hour,rides_previous_1342_hour,rides_previous_1341_hour,rides_previous_1340_hour,rides_previous_1339_hour,rides_previous_1338_hour,rides_previous_1337_hour,rides_previous_1336_hour,rides_previous_1335_hour,...,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_location_id,average_rides_last_8_weeks,hour,day_of_week
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,1,0.000,0,5
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1,0.000,0,6
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,2.0,0.0,2.0,0.0,0.0,0.0,1,0.000,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000,0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,1,0.000,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57500,7.0,5.0,1.0,1.0,3.0,4.0,15.0,17.0,21.0,9.0,...,7.0,3.0,5.0,2.0,8.0,5.0,265,4.000,0,0
57501,5.0,1.0,1.0,1.0,4.0,7.0,8.0,17.0,12.0,15.0,...,8.0,8.0,4.0,2.0,7.0,10.0,265,3.875,0,1
57502,6.0,3.0,3.0,1.0,0.0,3.0,7.0,20.0,12.0,27.0,...,7.0,2.0,2.0,6.0,6.0,3.0,265,4.125,0,2
57503,10.0,2.0,1.0,1.0,4.0,3.0,11.0,13.0,7.0,15.0,...,0.0,4.0,5.0,2.0,7.0,2.0,265,7.375,0,3


In [117]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_regression

pipeline = make_pipeline(
    add_feature_average_rides_last_8_weeks,
    add_temporal_features,
    SelectKBest(f_regression, k=40), #f_regression is for regression, #k=33 is the num of features
    lgb.LGBMRegressor()
)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_8_weeks at 0x16143e550>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('selectkbest',
                 SelectKBest(k=40,
                             score_func=<function f_regression at 0x15f6739d0>)),
                ('lgbmregressor', LGBMRegressor())])

In [118]:
predictions = pipeline.predict(X_test)

from sklearn.metrics import mean_absolute_error
test_mae_importance = mean_absolute_error(y_test, predictions)
print(f'{test_mae_importance=:.4f}')

test_mae_importance=2.5951


### Summary 16 weeks (4 months) with 70\30 split
[w\o feature importance] 
- mae = xxx
- feature_size = xxx
- pipeline train time = xxx

[w feature importance] 
- test_mae_importance = 2.59
- feature_size = 40
- pipeline train time = 3 sec